In [7]:
import nltk
import math
import os
import string
import spacy 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from  more_itertools import unique_everseen

nlp = spacy.load("en_core_web_sm")
#nltk.download('punkt') #Необходимо для первого запуска.


def make_tf_idf_list(dict, filelist):
    stop_words = set(stopwords.words("english"))
    for i in filelist:
        count_words = 0
        tf_idf_list = dict.copy()
        file = open(os.path.join('all_text/'+ i), 'r', encoding="utf-8")
        text = file.read()
        file.close()
        text = text.lower()
        r = re.compile("^[a-z]+$") 
        nltk_words = nltk.word_tokenize(text)
        nltk_words = [word for word in filter(r.match, nltk_words)] #Удаление не английских слов.
        nltk_words = [word for word in nltk_words if not word in stop_words] #Удаление стоп-слов.
        
        #Заполняем информацию о встречаемости слова в отдельном документе.
        for nltk_word in nltk_words:
            count_words += 1
            lemma = nlp(nltk_word)[0].lemma_
            tf_idf_list[lemma][0] += 1
        
        for term in dict:
            if tf_idf_list[term][0] == 0:
                del tf_idf_list[term] #Удаляем слова, которые не встречались в этом документе.
            else:
                tf_idf_list[term][0] = tf_idf_list[term][0] / count_words # Подсчёт tf.
                tf_idf_list[term][2] = tf_idf_list[term][0] * tf_idf_list[term][1] #Подсчёт tf-idf.
        file_num = re.findall(r'\d+', i)[0]
        file = open('all_tf_idf/tf_idf_{}.txt'.format(file_num), 'w', encoding="utf-8")
        for key,val  in tf_idf_list.items():
            file.write('\n{} {} {} '.format(key, val[1], val[2]))
        file.close()
        
if __name__ == "__main__":
    
    dir = 'all_text/'
    filelist = os.listdir(dir)
    dict = {}
    
    #Нахождение idf для каждого термина.
    f = open('invert_list.txt','r', encoding="utf-8")
    terms = f.readlines()
    f.close()
    for term in terms:
        arr = term.split(' ')
        dict[arr[0]] = [0, math.log(len(filelist)/(len(arr) - 2)), 0]
    #Создание списка.
    make_tf_idf_list(dict, filelist)    
    print("Done.")
       

Done.
